In [1]:
import pandas as pd
import minsearch
import os
from groq import Groq
from dotenv import load_dotenv
from tqdm.auto import tqdm


In [2]:
# Load environment variables from .env file
load_dotenv()
GROQ_API_KEY="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"

In [3]:
# Load environment variables from .env file
load_dotenv()
GROQ_API_KEY="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"

## Data Loading

In [4]:
file_path = 'data/data.csv'
df = pd.read_csv(file_path)
documents =df.to_dict(orient='records')

In [5]:
df.head()

,id,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instructions
0,0,Push-Ups,Strength,Bodyweight,Upper Body,Push,"Pectorals, Triceps, Deltoids",Start in a high plank position with your hands...
1,1,Squats,Strength,Bodyweight,Lower Body,Push,"Quadriceps, Glutes, Hamstrings",Stand with feet shoulder-width apart. Lower yo...
2,2,Plank,Strength/Mobility,Bodyweight,Core,Hold,"Rectus Abdominis, Transverse Abdominis",Start in a forearm plank position with your el...
3,3,Deadlift,Strength,Barbell,Lower Body,Pull,"Glutes, Hamstrings, Lower Back","Stand with feet hip-width apart, barbell in fr..."
4,4,Bicep Curls,Strength,Dumbbells,Upper Body,Pull,"Biceps, Forearms","Stand with a dumbbell in each hand, arms fully..."


In [5]:
documents[0]

{'id': 0,
 'exercise_name': 'Push-Ups',
 'type_of_activity': 'Strength',
 'type_of_equipment': 'Bodyweight',
 'body_part': 'Upper Body',
 'type': 'Push',
 'muscle_groups_activated': 'Pectorals, Triceps, Deltoids',
 'instructions': 'Start in a high plank position with your hands under your shoulders. Lower your body until your chest nearly touches the floor. Push back up to the starting position.'}

In [17]:
prompt_template = """
You emulate a user of our fitness assistant application.
Formulate 5 questions this user might ask based on a provided exercise.
Make the questions specific to this exercise.
The record should contain the answer to the questions, and the questions should
be complete and not too short. Use as fewer words as possible from the record. 

The record:

exercise_name: {exercise_name}
type_of_activity: {type_of_activity}
type_of_equipment: {type_of_equipment}
body_part: {body_part}
type: {type}
muscle_groups_activated: {muscle_groups_activated}
instructions: {instructions}

Provide the output in parsable JSON without using code blocks:

{{"questions": ["question1", "question2", ..., "question5"]}}
""".strip()

In [18]:
prompt = prompt_template.format(**documents[0])


In [13]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [32]:
def llm(prompt):
    response = client.chat.completions.create(
        model='mixtral-8x7b-32768',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [33]:
questions=llm(prompt)

In [35]:
json.loads(questions)

{'questions': ['What type of activity is a Push-Up considered?',
  'What equipment is needed for Push-Ups?',
  'Which part of the body does a Push-Up primarily target?',
  'What type of motion is involved in Push-Ups?',
  'What muscle groups are activated when doing Push-Ups?']}

In [36]:
def generate_questions(doc):
    prompt = prompt_template.format(**doc)

    response = client.chat.completions.create(
        model='mixtral-8x7b-32768',
        messages=[{"role": "user", "content": prompt}]
    )

    json_response = response.choices[0].message.content
    return json_response

In [38]:
for doc in tqdm(documents): 
    doc_id = doc['id']
    if doc_id in results:
        continue

    questions_raw = generate_questions(doc)
    
    # Print or log the raw data for inspection
    print(f"Raw JSON data for doc_id {doc_id}: {questions_raw}")
    
    try:
        questions = json.loads(questions_raw)
        results[doc_id] = questions['questions']
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError for doc_id {doc_id}: {e}")
        # Handle the error or skip the document
        continue


  0%|          | 0/207 [00:00<?, ?it/s]

Raw JSON data for doc_id 19: {
"questions": [
"What exercise is called Goblet Squats and what type of activity is it?",
"What equipment is needed for Goblet Squats?",
"What part of the body does Goblet Squats primarily target?",
"What type of movement is Goblet Squats, push or pull?",
"What are the steps to perform Goblet Squats correctly?"
]
}
Raw JSON data for doc_id 20: {
"questions": [
"What is the name of the exercise that targets the glutes and hamstrings?",
"What type of activity is the Glute Bridge?",
"What equipment is needed for this exercise?",
"Which part of the body does the Glute Bridge focus on?",
"What type of motion is involved in the Glute Bridge?",
"What are the steps to perform the Glute Bridge correctly?"
]
}
Raw JSON data for doc_id 21: {
"questions": [
"What is the name of the exercise that targets the triceps using dumbbells?",
"What type of activity is the Overhead Tricep Extension?",
"What equipment do I need to perform this exercise?",
"Which part of the body

In [40]:
final_results = []

for doc_id, questions in results.items():
    for q in questions:
        final_results.append((doc_id, q))

In [41]:
final_results[0]

(0, 'What type of activity is a push-up exercise?')

In [43]:
df_results=pd.DataFrame(final_results,columns=["id","questions"])

In [46]:
df_results.head()

,id,questions
0,0,What type of activity is a push-up exercise?
1,0,What equipment is needed for push-ups?
2,0,Which part of the body does the push-up exerci...
3,0,"What type of push-up is this, and what is its ..."
4,0,Which muscle groups are activated during push-...


In [48]:
df_results.to_csv('data/ground-truth-retrieval.csv', index=False)


In [6]:
rf =pd.read_csv('data/ground-truth-retrieval.csv')

In [7]:
df

,id,exercise_name,type_of_activity,type_of_equipment,body_part,type,muscle_groups_activated,instructions
0,0,Push-Ups,Strength,Bodyweight,Upper Body,Push,"Pectorals, Triceps, Deltoids",Start in a high plank position with your hands...
1,1,Squats,Strength,Bodyweight,Lower Body,Push,"Quadriceps, Glutes, Hamstrings",Stand with feet shoulder-width apart. Lower yo...
2,2,Plank,Strength/Mobility,Bodyweight,Core,Hold,"Rectus Abdominis, Transverse Abdominis",Start in a forearm plank position with your el...
3,3,Deadlift,Strength,Barbell,Lower Body,Pull,"Glutes, Hamstrings, Lower Back","Stand with feet hip-width apart, barbell in fr..."
4,4,Bicep Curls,Strength,Dumbbells,Upper Body,Pull,"Biceps, Forearms","Stand with a dumbbell in each hand, arms fully..."
...,...,...,...,...,...,...,...,...
202,202,Incline Dumbbell Row,Strength,Dumbbells,Upper Body,Pull,"Latissimus Dorsi, Biceps",Lie face down on an incline bench with a dumbb...
203,203,Machine Lat Pulldown,Strength,Machine,Upper Body,Pull,"Latissimus Dorsi, Biceps",Sit at a lat pulldown machine with a wide grip...
204,204,One-Arm Cable Row,Strength,Cable Machine,Upper Body,Pull,"Latissimus Dorsi, Biceps",Stand facing a cable machine with the handle a...
205,205,Face Pull,Strength,Cable Machine,Upper Body,Pull,"Rear Deltoids, Trapezius, Rhomboids",Attach a rope handle to a high cable pulley. P...


In [9]:
rf.head(10)

,id,questions
0,0,What type of activity is a push-up exercise?
1,0,What equipment is needed for push-ups?
2,0,Which part of the body does the push-up exerci...
3,0,"What type of push-up is this, and what is its ..."
4,0,Which muscle groups are activated during push-...
5,1,What type of exercise are Squats?
6,1,What equipment is needed to perform Squats?
7,1,What part of the body do Squats primarily target?
8,1,What type of movement is involved in Squats?
9,1,What muscle groups are engaged during Squats a...
